# Setup the git project

# 1 - Connect to Vantage

In [19]:
import teradataml as tdml
tdml.options.configure.byom_install_location = "mldb"
tdml.configure.val_install_location = 'TRNG_XSP'
tdml.display.print_sqlmr_query = True
import getpass
import json
from datetime import datetime
tic = datetime.now()
tdml.__version__

'17.20.00.04'

In [2]:
Param = {
    'host'          : 'tdprd2.td.teradata.com', 
    'user'          : 'dm250067', 
    'password'      : "ENCRYPTED_PASSWORD(file:{},file:{})".format ('../../PassKey.properties','../../EncPass.properties'), #getpass.getpass(), 
    'logmech'       : 'LDAP',
    'database'      : 'ADLSLSEMEA_DEMO_BANKING',
    'temp_database_name' : 'ADLSLSEMEA_DEMO_BANKING'
    }

Param = {
    'host'          : 'tdprd3.td.teradata.com', 
    'user'          : 'dm250067', 
    'password'      : "ENCRYPTED_PASSWORD(file:{},file:{})".format ('../../PassKey.properties','../../EncPass.properties'), #getpass.getpass(), 
    'logmech'       : 'LDAP',
    'database'      : 'ADLDSD_CHURN',
    'temp_database_name' : 'ADLDSD_CHURN'
    }

tdml.create_context(**Param)

C:\Users\dm250067\Anaconda3\envs\vantage39\lib\site-packages\teradataml\context\context.py:480: TeradataMlRuntimeWarning: Warning: Password is URL encoded.
  warnings.warn("Warning: Password is URL encoded.", category=TeradataMlRuntimeWarning)


Engine(teradatasql://:***@tdprd3.td.teradata.com/?DATABASE=ADLDSD_CHURN&LOGDATA=%2A%2A%2A&LOGMECH=%2A%2A%2A&USER=DM250067)

## 2 - Creating the model table

In [3]:
#ddl for Aoa_Byom_Models 
query = f"""CREATE SET TABLE {Param['database']}.Aoa_Byom_Models 
     (
      model_version VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      model_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      model_type VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      project_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      deployed_at TIMESTAMP(6) DEFAULT CURRENT_TIMESTAMP(6),
      model BLOB(2097088000))
UNIQUE PRIMARY INDEX ( model_version );
"""
 
try:
    tdml.execute_sql(query)
except:
    tdml.execute_sql(f"DROP TABLE {Param['database']}.Aoa_Byom_Models;")
    tdml.execute_sql(query)

## 3 - Creating the aoa statistics table

In [4]:


#ddl for Aoa_Byom_Models 
query = f"""CREATE MULTISET TABLE {Param['database']}.aoa_statistics_metadata ,FALLBACK ,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO,
     MAP = TD_MAP1
     (
      column_name VARCHAR(1024) CHARACTER SET LATIN NOT CASESPECIFIC,
      column_type VARCHAR(1024) CHARACTER SET LATIN NOT CASESPECIFIC,
      stats VARCHAR(1024) CHARACTER SET LATIN NOT CASESPECIFIC,
      update_ts VARCHAR(1024) CHARACTER SET LATIN NOT CASESPECIFIC)
NO PRIMARY INDEX ;
"""
 
try:
    tdml.execute_sql(query)
except:
    tdml.execute_sql(f"DROP TABLE {Param['database']}.aoa_statistics_metadata;")
    tdml.execute_sql(query)

## 4 - Creating the prediction table

In [6]:
#ddl for Pima_Patient_Predictions
query = f"""CREATE MULTISET TABLE {Param['database']}.Transaction_fraud_predictions 
     (
      job_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      txn_id BIGINT,
      isFraud BIGINT,
      json_report CLOB(1048544000) CHARACTER SET LATIN)
PRIMARY INDEX ( job_id );;
"""
 
try:
    tdml.execute_sql(query)
except:
    tdml.execute_sql(f"DROP TABLE {Param['database']}.Transaction_fraud_predictions;")
    tdml.execute_sql(query)

In [11]:
%%writefile script.bat
aoa feature create-stats-table -m ADLDSD_CHURN.transactions
aoa feature compute-stats -s ADLDSD_CHURN.PIMA -m ADLDSD_CHURN.ADLDSD_CHURN -t continuous -c amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest

Overwriting script.bat


In [12]:
import aoa

In [13]:
help(aoa.create_features_stats_table)

Help on function create_features_stats_table in module aoa.stats.store:

create_features_stats_table(features_table: str) -> None



In [51]:
metadata_table = "ADLDSD_CHURN.transactions_features"
tdml.db_drop_table(table_name=metadata_table.split('.')[1], schema_name=metadata_table.split('.')[0])
aoa.create_features_stats_table(metadata_table)

In [53]:
fs = aoa.compute_categorical_stats(features_df=tdml.DataFrame(tdml.in_schema(Param['database'],'transactions_fraud')), categorical_features = ['transaction_type','isfraud'])
aoa.stats.store.save_feature_stats(features_table=metadata_table, feature_type="categorical", stats=fs)

In [54]:
fs = aoa.compute_continuous_stats(features_df=tdml.DataFrame(tdml.in_schema(Param['database'],'transactions_fraud')), continuous_features = [c.lower() for c in ['amount', 'oldbalanceOrig', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']])
aoa.stats.store.save_feature_stats(features_table=metadata_table, feature_type="continuous", stats=fs)

In [16]:
help(aoa.compute_continuous_stats)

Help on function compute_continuous_stats in module aoa.stats.stats:

compute_continuous_stats(features_df: teradataml.dataframe.dataframe.DataFrame, continuous_features: List)



In [29]:
help(aoa.compute_categorical_stats)

Help on function compute_categorical_stats in module aoa.stats.stats:

compute_categorical_stats(features_df: teradataml.dataframe.dataframe.DataFrame, categorical_features: List)



In [22]:
tdml.DataFrame(tdml.in_schema(Param['database'],'transactions')).columns

['type',
 'txn_id',
 'step',
 'amount',
 'nameOrig',
 'oldbalanceOrig',
 'newbalanceOrig',
 'nameDest',
 'oldbalanceDest',
 'newbalanceDest',
 'isFraud',
 'isFlaggedFraud',
 'fold']

In [23]:
tdml.DataFrame(tdml.in_schema(Param['database'],'transactions')).tdtypes

COLUMN NAME,TYPE
type,"VARCHAR(length=11, charset='LATIN')"
txn_id,INTEGER()
step,SMALLINT()
amount,FLOAT()
nameOrig,"VARCHAR(length=17, charset='LATIN')"
oldbalanceOrig,FLOAT()
newbalanceOrig,FLOAT()
nameDest,"VARCHAR(length=16, charset='LATIN')"
oldbalanceDest,FLOAT()
newbalanceDest,FLOAT()


In [34]:
tdml.DataFrame(tdml.in_schema(Param['database'],'transactions'))

type,txn_id,step,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,fold
PAYMENT,7620262,332,5718.81,C2141361931,0.0,0.0,M1199192088,0.0,0.0,0,0,test
CASH_OUT,28581315,308,163379.24,C662117102,5453.0,0.0,C1005617219,0.0,163379.24,0,0,test
PAYMENT,38587780,325,8054.09,C1526980821,0.0,0.0,M1338340859,0.0,0.0,0,0,test
CASH_OUT,34577910,396,39646.41,C1017287590,197.0,0.0,C1468150814,0.0,39646.41,0,0,test
CASH_OUT,13622424,162,170837.06,C1937763342,354146.0,183308.94,C363333681,150022.95,320860.02,0,0,test
PAYMENT,4606538,355,8405.87,C2082013906,183.0,0.0,M617728745,0.0,0.0,0,0,train
CASH_OUT,8616408,139,209704.11,C1723329633,119858.0,0.0,C768750011,1559650.64,1769354.75,0,0,test
PAYMENT,37597201,383,13978.9,C207869969,18904.0,4925.1,M1702500665,0.0,0.0,0,0,test
TRANSFER,16636148,42,1354494.52,C994829701,103160.0,0.0,C729279544,3685023.08,5039517.6,0,0,train
CASH_IN,2614246,324,192756.85,C1682449475,1505516.49,1698273.34,C1602525239,342362.63,149605.78,0,0,train


In [43]:
df = tdml.DataFrame(tdml.in_schema(Param['database'],'transactions'))
df

type,txn_id,step,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,fold
CASH_OUT,39593918,204,212248.27,C1849762667,0.0,0.0,C262877020,13651578.46,13863826.73,0,0,test
PAYMENT,36599913,213,8870.93,C1594622133,24943.0,16072.07,M1631347630,0.0,0.0,0,0,test
TRANSFER,38611924,35,1340150.54,C133428518,42740.0,0.0,C1936349546,53820.17,1393970.72,0,0,test
CASH_OUT,39608070,331,48309.37,C501560352,0.0,0.0,C1555244133,962116.37,1010425.75,0,0,test
PAYMENT,7591285,135,3411.04,C762014588,5034.0,1622.96,M923520755,0.0,0.0,0,0,train
CASH_IN,12597301,135,165576.42,C1865414933,7676207.1,7841783.52,C134004657,2062008.05,1896431.63,0,0,train
CASH_OUT,16607171,259,47333.67,C329872725,49893.0,2559.33,C444599139,196310.69,243644.36,0,0,train
PAYMENT,35584048,188,8936.1,C1533849896,451.0,0.0,M1349394037,0.0,0.0,0,0,test
CASH_IN,19589573,348,246966.29,C126110896,1020.0,247986.29,C876900970,0.0,0.0,0,0,test
CASH_OUT,14583557,644,89046.73,C1737262733,37792.0,0.0,C2010914388,440602.75,529649.48,0,0,train


In [46]:
df_new = df.assign(transaction_type = df['type'])
df_new = df_new[['txn_id','amount','nameOrig','oldbalanceOrig','newbalanceOrig','nameDest','oldbalanceDest','newbalanceDest','transaction_type','isFraud','fold']]
df_new

txn_id,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,transaction_type,isFraud,fold
4621159,3668.92,C417453074,50323.0,46654.08,C563580064,0.0,3668.92,CASH_OUT,0,train
10623321,4545.32,C1016540797,176876.27,172330.95,M1593161345,0.0,0.0,PAYMENT,0,test
1607435,9634.38,C534106108,21829.0,12194.62,M11930111,0.0,0.0,PAYMENT,0,train
15629337,153114.1,C715518072,593.0,153707.1,C1469728600,170438.67,17324.57,CASH_IN,0,train
2617733,11919.55,C1686741077,35490.0,23570.45,M1180662645,0.0,0.0,PAYMENT,0,train
22582640,78297.72,C542557550,7405754.44,7484052.15,C1236801109,536474.95,458177.23,CASH_IN,0,test
11633619,13067.92,C1476331520,0.0,0.0,C1494544060,6095824.35,6108892.27,TRANSFER,0,test
9627175,137425.82,C309788431,426.0,137851.82,C863042157,1018064.5,880638.68,CASH_IN,0,test
34612250,13847.97,C403858928,143952.0,130104.03,M198272800,0.0,0.0,PAYMENT,0,test
25576645,43546.47,C538767247,11640.0,0.0,C1535960803,152338.5,195884.97,CASH_OUT,0,test


In [47]:
df_new.to_sql(table_name = 'transactions_fraud', schema_name= Param['database'], primary_index = 'txn_id')